In [1]:
import os
import torch
from torchvision.io import read_image
from torch.utils import data
from torchvision.transforms.functional import convert_image_dtype
import cv2
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
class GTA5DataSet(data.Dataset):
  def __init__(self, dataset_directory = None, img_directory = None, labels_directory = None, name_list = None, image_dimensions = (1024, 512)):
    self.name_list = name_list
    self.dataset_directory = dataset_directory
    self.file_name_list =  [file_name.strip() for file_name in open(os.path.join(dataset_directory, name_list))]
    self.img_directory = img_directory
    self.labels_directory = labels_directory
    self.image_dimensions = image_dimensions
    self.label_map = {7: 0, 8: 1, 11: 2, 12: 3, 13: 4, 17: 5,
                      19: 6, 20: 7, 21: 8, 22: 9, 23: 10, 24: 11, 25: 12,
                      26: 13, 27: 14, 28: 15, 31: 16, 32: 17, 33: 18}


  def __len__(self):
    return len(self.file_name_list)

  def __getitem__(self, position):
    img_path = os.path.join(self.img_directory, self.file_name_list[position])
    label_path = os.path.join(self.labels_directory, self.file_name_list[position].replace("_leftImg8bit", "_gtFine_labelIds"))

    #we were reading image directly into tensor, which is wrong. We should read it as array and possibly the DataLoader function will auto convert it into tensor
    #professor suggested using pillow for this part

    #here we read the image and convert it to RGB from BGR
    image_RGB = np.asarray(Image.open(img_path).convert("RGB").resize(self.image_dimensions,  Image.BICUBIC), np.float32)
    image_label = np.asarray(Image.open(label_path).resize(self.image_dimensions,  Image.NEAREST), np.float32)

    size = image_RGB.shape
    image_RGB = image_RGB[:, :, ::-1]  # change to BGR
    image_RGB -= (128,128,128)
    image_RGB = image_RGB.transpose((2, 0, 1))

    #remap label pixels
    
    label_RGB_copy = 255 * np.ones((self.image_dimensions[1], self.image_dimensions[0]), dtype=np.float32)
    for id, value in self.label_map.items():
      label_RGB_copy[image_label == id] = value

    return image_RGB.copy(), label_RGB_copy.copy()

In [3]:
from google.colab import drive
import argparse

drive.mount('/content/drive/')

#change the directory manually if you are not marco or allan
who_is_accesing = input("say your name ")
if who_is_accesing == "marco":
  %cd "/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3"
  directory  = "/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3"
elif who_is_accesing == "allan":
  %cd "/content/drive/MyDrive/Summer 2022/task_3"
  directory = "/content/drive/MyDrive/Summer 2022/task_3"
elif who_is_accesing == "mldl":
  %cd "/content/drive/MyDrive/task_3"
  directory = "/content/drive/MyDrive/task_3"
  
from loss import find_frequency_class
  



dataset_source = GTA5DataSet(directory+"/dataset/GTA5", os.path.join(directory+"/dataset/GTA5", "images"), os.path.join(directory+"/dataset/GTA5", "labels"), "train.txt", image_dimensions = (512,1024))
dataloader_source = DataLoader(dataset_source, batch_size = 8, num_workers = 2, shuffle = True, drop_last= True, persistent_workers= True)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
say your name marco
/content/drive/MyDrive/Fourth Year/Machine learning and Deep learning Course/task_3


In [4]:
dic = find_frequency_class(dataloader_source)

In [11]:
print(type(list(dic.values())))

<class 'list'>


In [29]:
def get_weights_classes(number_classes = 19, samples_per_class = None, power = 1 ):
  w = 1.0/np.array(np.power(samples_per_class, power))
  w2 = w/np.sum(w) * number_classes
  return w2


In [30]:
w = get_weights_classes(19, list(dic.values()),1)

In [31]:
print(w)

[ 0.08315246  0.08611596  0.08281717  0.08721298  0.09874355  0.08298448
  0.11538572  0.08349048  0.08539982  0.08298448  0.12485506  0.1521382
  0.08891194  0.61309426  0.09335753  0.42347748  0.50712735  2.41631266
 13.69243842]
